In [5]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings

import joblib
from utils import make_dataset, display_results

warnings.filterwarnings('ignore')

TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


### Disparate Impact Remover

In [8]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

dataset = make_dataset(data, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)

disp_imp_remover = DisparateImpactRemover(sensitive_attribute='race',repair_level=1.0)

dataset = disp_imp_remover.fit_transform(dataset).convert_to_dataframe()[0]

In [9]:
data_cleaned = data.copy()
data_cleaned['race'] = dataset['race'].values

In [10]:
train, test = train_test_split(data_cleaned, test_size=0.2, random_state=1234)

X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [11]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)

y_test_pred = clf.predict_proba(X_test)
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

roc_auc = roc_auc_score(y_test, y_test_pred[:,1])

In [12]:
ground_truth_test = make_dataset(test, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test, 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

# Reweighing

In [13]:
from aif360.algorithms.preprocessing import Reweighing

reweighing = Reweighing(**PRIVILEGED_INFO)

In [14]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

train_dataset = make_dataset(train, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
test_dataset = make_dataset(test, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)

train_dataset = reweighing.fit_transform(train_dataset)
test_dataset = reweighing.transform(test_dataset)

In [15]:
train_dataset.instance_weights

array([1.06614556, 1.06614556, 0.9307455 , ..., 1.06614556, 0.89397015,
       0.9307455 ])

In [16]:
train['race']=train_dataset.convert_to_dataframe()[0]['race'].values
test['race']=test_dataset.convert_to_dataframe()[0]['race'].values

train_weights = train_dataset.instance_weights

In [17]:
X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [18]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train, sample_weight=train_weights)

y_test_pred = clf.predict_proba(X_test)
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

roc_auc = roc_auc_score(y_test, y_test_pred[:,1])

In [19]:
ground_truth_test = make_dataset(test, 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test, 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [20]:
joblib.dump((clf_metric,roc_auc), '../results/1.2-preprocessing.pkl')
display_results('../results/1.2-preprocessing.pkl')

,metric_names,scores
0,roc_auc_score,0.727046
1,true_positive_rate_difference,-0.087543
2,false_positive_rate_difference,-0.193763
3,false_omission_rate_difference,-0.022008
4,false_discovery_rate_difference,-0.023299
5,error_rate_difference,-0.019783
6,false_positive_rate_ratio,0.690940
7,false_negative_rate_ratio,1.610615
8,false_omission_rate_ratio,0.938378
9,false_discovery_rate_ratio,0.930680


# LFR

# OPTIM